# Real-world example: calculating the potential yield of maize on land that is currently forests in Rwanda

Now that we know how to open, manipulate, and save rasters, we can do some real-world analysis. We're going to calculate the potential yield of maize on land that is currently forests in Rwanda. This might be relevant to, for instance, a regression you would do on household data and whether individuals are likely to cut down their forests to plant maize.

But let's look at these two files in QGIS. The easiest way to load them into QGIS, in my opinion, is to drag the raster file from your file explorer onto QGIS.

What we see is not super pretty because we need to modify the colorbar.

![](images/paste-4.png)

## Double-click the layer you added to bring up its properties. Then select the Symbology tab on the left.

![](images/paste-8.png)

## Choose a colorbar you like. Ensure the min-max values are set.

![](images/paste-9.png)

## This looks a little better

![](images/paste-10.png)

## Now add the Rwanda LULC map you loaded earlier

-   It will now show up in the Layers menu

    -   Note that only the topmost checked layer will be displayed on top, covering other things.

    -   Scroll to Rwanda

![](images/paste-12.png)

## We have several visualization problems

-   First, there are very low maize yields in the area, and so they're almost the lightest color.

    -   Adjust the maximium of the colorbar to \~1000 so you can see the variation

-   Second, these two rasters are at very different resolutions!

![](images/paste-14.png)

Any raster-math we do on them will either fail or (worse) succeed at producing nonsense. This is because the underlying numpy arrays are not representing the same part of earth at the same resolution. We will need to do two things first: **clip** and **reproject** one of the layers to match the other.

## Clip

-   In QGIS, Find the command Clip Raster by Extent.

![](images/paste-16.png)

-   This will bring up a Window to launch the tool.

-   Set the Input layer as the Maize layer (this is the one we will clip to be smaller)

-   Set the Clipping Extent to match the Rwanda LULC map.

![](images/paste-17.png)

## The Clipped map doesn't cover all of the LULC!

-   If you thought data cleaning was hard on tabular data...

![](images/paste-21.png)

-   There are tons of ways to fix this in QGIS, but we're actually going to go back to Python to harness the power of a fun new library: Pygeoprocessing.

## Reprojection

-   In QGIS, Find the raster Warp command.

![](images/paste-15.png)

## Pygeoprocessing

-   Pygeoprocessing is a python library that is built on top of GDAL. It is a little more user-friendly and has some nice features.
-   We are going to use the align_and_resize_raster_stack() function.
    -   But, how do we know what to input into this function?
        -   Let's inspect it.
        -   Put your cursor on `align_and_resize_raster_stack` in the code block below and press F-12. Your computer might not access the Function keys automatically (instead overwriting it with some media player nonsense or the like), so you might have to find the Fn button to hold at the same time (this is switchable).

In [ ]:
import pygeoprocessing 

pygeoprocessing.align_and_resize_raster_stack

This will bring you straight to the function that you installed via Mamba! It will have extensive documentation.

![](images/paste-22.png)

In [ ]:
# Redefine these, just to be safe
yield_file_path = os.path.join(data_directory, yield_filename)
lulc_file_path = os.path.join(data_directory, lulc_filename)

# Define some new paths to store the aligned and clipped files
yield_aligned_file_path = os.path.join(data_directory, 'yield_aligned.tif')
lulc_aligned_file_path = os.path.join(data_directory, 'lulc_aligned.tif')

# Configure the paths into lists (this helps the function generalize to MANY files)
base_raster_path_list = [yield_file_path, lulc_file_path]
target_raster_path_list = [yield_aligned_file_path, lulc_aligned_file_path]

# Create a list as long as the above with the resampling for each respective layer. There's Much more to discuss here
# but for now we'll just use bilinear
resample_method_list = ['bilinear', 'bilinear']

# Next decide which area all the layers will be clipped to. Intersection means it will just be the ones that overlap.
bounding_box_mode = 'intersection'

# Decide which of the base_rasters should define the resolution and projection 
# of the output data. For us, we want it to be the resolution of the LULC map, which is 
# in the second position of the list (index 1)
raster_align_index = 1

# Finally, we need to set the desired pixelsize the data will be resampled to.
target_pixel_size = (0.000833333333333333, -0.000833333333333333)

pygeoprocessing.align_and_resize_raster_stack(base_raster_path_list, target_raster_path_list, resample_method_list,
        target_pixel_size, bounding_box_mode, 
        raster_align_index=raster_align_index)

## When this first worked for me, I almost cried

Seriously. 

We'll learn more about this package soon.

The key element now is that each of the aligned files has exactly the same resolution and extent, which lets us do raster math on them super easily because for any for any range of row-columns, the values are representing the same place on earth.

But for now, let's load them up in QGIS.
